In [11]:
import cv2
import os
import numpy as np
from tqdm import tqdm
import glob
import torch
import pandas as pd
import gc
from os.path import join as join_path
from PIL import Image
from torchvision import transforms

In [ ]:
PATH_TO_FOLDER = '/iraa/main'

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')

# Init model

In [14]:
os.chdir(join_path(PATH_TO_FOLDER, 'adversarial_attacks', 'facpa'))
from cnn_attack_inference import UnetGenerator, normalize_and_scale

In [15]:
def add_delta_cnn(image: torch.tensor, delta_im, h, w, C=10):
    delta_im = normalize_and_scale(delta_im, C) 
    delta_im = delta_im.squeeze().data.cpu().numpy().transpose(1, 2, 0) 
    delta = np.tile(delta_im,(image.shape[0] // h + 1, image.shape[1] // w + 1, 1))[:image.shape[0], :image.shape[1], :]
    out = image + delta
    out[out > 1] = 1
    out[out < 0] = 0
    res_img = (out * 255).astype('uint8')
    return res_img

In [16]:
RESIZE_PARAM = 256

cnn_resize = transforms.Resize((512, 1024))
cnn_resize_256 = transforms.Resize((256))
to_tensor = transforms.ToTensor()

# Run IQA models

In [20]:
all_images = sorted(glob.glob(join_path(PATH_TO_FOLDER, 'data', 'KADID10K/kadid10k/images', '*')))
len(all_images)

10206

## Linearity

In [ ]:
from torchvision.transforms.functional import resize, to_tensor, normalize
from PIL import Image

In [22]:
netG = UnetGenerator(3, 3, 64, norm_type='instance', act_type='relu').to(device)
netG.load_state_dict(torch.load(join_path(PATH_TO_FOLDER, 'adversarial_attacks', 'facpa', 'weigths', 'unet_lin.pth'), map_location=device))
netG.eval();

In [24]:
os.chdir(join_path(PATH_TO_FOLDER, 'nr_iqa_metrics', 'Linearity'))
from IQAmodel import IQAModel

In [ ]:
model = IQAModel().to(device)
model_path = '/home/jovyan/storage/NR-metric-models/p1q2.pth'
checkpoint = torch.load(model_path)
k = checkpoint['k']
b = checkpoint['b']
model.load_state_dict(checkpoint['model'])
model.eval();

In [26]:
C = 22

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    im = Image.open(image_file).convert('RGB')
    im = resize(im, (498, 664))
    im = to_tensor(im).to(device)
    im = normalize(im, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    with torch.no_grad():
        y = model(im.unsqueeze(0))[-1].cpu().detach().item()
        before_score = y * k[-1] + b[-1]
    
    im = cv2.imread(image_file)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype('float32') / 255.
    
    h, w = im.shape[0] // RESIZE_PARAM, im.shape[1] // RESIZE_PARAM
    if h == 0 or w == 0:
        image = cnn_resize_256(transforms.ToTensor()(im))
        h, w = image.shape[1] // RESIZE_PARAM, image.shape[2] // RESIZE_PARAM
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM
        image = image[:, :h, :w]
    else:
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM

        im64 = im[:h, :w, :]
        image = transforms.ToTensor()(im64)
   
    delta_im = netG(image.unsqueeze(0).to(device))
    image_after_attack = add_delta_cnn(im, delta_im, h, w, C=C)
    
    im_pil = Image.fromarray(cv2.cvtColor(image_after_attack, cv2.COLOR_BGR2RGB))
    im = im_pil.convert('RGB')
    im = resize(im, (498, 664))
    im = to_tensor(im).to(device)
    im = normalize(im, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    with torch.no_grad():
        y = model(im.unsqueeze(0))[-1].cpu().detach().item()
        after_score = y * k[-1] + b[-1]
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'cnn_{C}_255'
df['metric'] = 'lin'

torch.cuda.empty_cache()
gc.collect()

  1%|███                                                                                                                                                                                                                                                                 | 118/10206 [08:17<12:08:16,  4.33s/it]

## Paq-2-Piq

In [ ]:
netG = UnetGenerator(3, 3, 64, norm_type='instance', act_type='relu').to(device)
netG.load_state_dict(torch.load(join_path(PATH_TO_FOLDER, 'adversarial_attacks', 'facpa', 'weigths', 'unet_p2p.pth'), map_location=device))
netG.eval();

In [ ]:
os.environ['my_device'] = str(device)

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'nr_iqa_metrics', 'Paq-2-Piq'))
from paq2piq_standalone import InferenceModel, RoIPoolModel

In [ ]:
model_path = join_path(PATH_TO_FOLDER, 'nr_iqa_metrics', 'Paq-2-Piq', 'RoIPoolModel.pth')
model = InferenceModel(RoIPoolModel().to(device), model_path)

In [ ]:
C = 8

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    im = cv2.imread(image_file)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    before_score = model.predict(im)['global_score']
    im = im.astype('float32') / 255.
    
    h, w = im.shape[0] // RESIZE_PARAM, im.shape[1] // RESIZE_PARAM
    if h == 0 or w == 0:
        image = cnn_resize_256(transforms.ToTensor()(im))
        h, w = image.shape[1] // RESIZE_PARAM, image.shape[2] // RESIZE_PARAM
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM
        image = image[:, :h, :w]
    else:
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM

        im64 = im[:h, :w, :]
        image = transforms.ToTensor()(im64)
   
    delta_im = netG(image.unsqueeze(0).to(device))
    image_after_attack = add_delta_cnn(im, delta_im, h, w, C=C)
    after_score = model.predict(image_after_attack)['global_score']
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'cnn_{C}_255'
df['metric'] = 'p2p'

In [ ]:
torch.cuda.empty_cache()
gc.collect()

## SPAQ

In [ ]:
from PIL import Image
import torchvision
from torchvision import transforms

In [ ]:
netG = UnetGenerator(3, 3, 64, norm_type='instance', act_type='relu').to(device)
netG.load_state_dict(torch.load(join_path(PATH_TO_FOLDER, 'adversarial_attacks', 'facpa', 'weigths', 'unet_spaq.pth'), map_location=device))
netG.eval();

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'nr_iqa_metrics', 'SPAQ'))

from Prepare_image import Image_load


class Baseline(torch.nn.Module):
	def __init__(self):
		super(Baseline, self).__init__()
		self.backbone = torchvision.models.resnet50(pretrained=False)
		fc_feature = self.backbone.fc.in_features
		self.backbone.fc = torch.nn.Linear(fc_feature, 1, bias=True)

	def forward(self, x):
		result = self.backbone(x)
		return result

model = Baseline()

model_path = '/home/jovyan/storage/NR-metric-models/BL_release.pt'

checkpoint = torch.load(model_path, device)
model.load_state_dict(checkpoint['state_dict'])
model.to(device);
model.eval();

prepare_image = Image_load(size=512, stride=224)

to_tensor = transforms.ToTensor()

In [ ]:
C = 40

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    try:
        img = prepare_image(Image.open(image_file).convert('RGB')).to(device)
    except:
        res.append([image_file.split('/')[-1], -1, -1])
        continue
    
    with torch.no_grad():
        before_score = model(img).mean().detach().cpu().item()
    
    im = cv2.imread(image_file)
    im = im.astype('float32') / 255.
    
    h, w = im.shape[0] // RESIZE_PARAM, im.shape[1] // RESIZE_PARAM
    if h == 0 or w == 0:
        image = cnn_resize_256(transforms.ToTensor()(im))
        h, w = image.shape[1] // RESIZE_PARAM, image.shape[2] // RESIZE_PARAM
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM
        image = image[:, :h, :w]
    else:
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM

        im64 = im[:h, :w, :]
        image = transforms.ToTensor()(im64)
   
    delta_im = netG(image.unsqueeze(0).to(device))
    image_after_attack = add_delta_cnn(im, delta_im, h, w, C=C)
    
    img = prepare_image(Image.fromarray(image_after_attack).convert('RGB')).to(device)
    with torch.no_grad():
        after_score = model(img).mean().detach().cpu().item()
    
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'cnn_{C}_255'
df['metric'] = 'spaq'

## KonCept512

In [ ]:
import torch.nn as nn
import torchvision
from PIL import Image
import argparse
import os
from torchvision.transforms import ToTensor, Resize, Normalize
from torchvision import transforms

In [ ]:
netG = UnetGenerator(3, 3, 64, norm_type='instance', act_type='relu').to(device)
netG.load_state_dict(torch.load(join_path(PATH_TO_FOLDER, 'adversarial_attacks', 'facpa', 'weigths', 'unet_koncept.pth'), map_location=device))
netG.eval();

In [ ]:
os.chdir(join_path(PATH_TO_FOLDER, 'nr_iqa_metrics', 'KonCept512'))

from inceptionresnetv2 import inceptionresnetv2

In [ ]:
class model_qa(nn.Module):
    def __init__(self,num_classes,**kwargs):
        super(model_qa,self).__init__()
        base_model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')
        self.base = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(1536, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),         
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self,x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x    

In [ ]:
KonCept512 = model_qa(num_classes=1)
KonCept512.load_state_dict(torch.load('/home/jovyan/storage/NR-metric-models/KonCept512.pth'))
KonCept512.eval().to(device);

In [ ]:
koncept_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((384, 512)),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

In [ ]:
def img_to_batch_koncept(img):
    batch_size_cur = 1
    img_batch = torch.zeros(1, 3, 384, 512).to(device) 
    img_batch[0]  = koncept_transform(img) 
    # for i in range(batch_size_cur):  
    #     img_batch[i] = koncept_transform(img) 
    return img_batch

In [ ]:
C = 24

In [ ]:
res = []
gains = []
for image_file in tqdm(all_images):
    image = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)
    with torch.no_grad():
        before_score = KonCept512(img_to_batch_koncept(image).to(device)).detach().detach().cpu().numpy()[0][0]

    im = image.astype('float32') / 255.
    h, w = im.shape[0] // RESIZE_PARAM, im.shape[1] // RESIZE_PARAM
    if h == 0 or w == 0:
        image = cnn_resize_256(to_tensor(im))
        h, w = image.shape[1] // RESIZE_PARAM, image.shape[2] // RESIZE_PARAM
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM
        image = image[:, :h, :w]
    else:
        h, w = h * RESIZE_PARAM, w * RESIZE_PARAM
        im64 = im[:h, :w, :]
        image = to_tensor(im64)
   
    delta_im = netG(image.unsqueeze(0).to(device))
    image_after_attack = add_delta_cnn(im, delta_im, h, w, C=C)

    delta_im = netG(image.unsqueeze(0).to(device))
    image_after_attack = add_delta_cnn(im, delta_im, h, w, C=C)

    with torch.no_grad():
        after_score = KonCept512(img_to_batch_koncept(image_after_attack).to(device)).detach().detach().cpu().numpy()[0][0]
    
        
    gains.append([before_score, after_score])
    res.append([image_file.split('/')[-1], before_score, after_score])
gains = np.array(gains)

df = pd.DataFrame(res, columns=['image_name', 'before_score', 'after_score'])
df['attack_type'] = f'cnn_{C}_255'
df['metric'] = 'koncept'